In [1]:
import os, gams, pandas as pd, numpy as np, gams2numpy, shutil,pickle,pyDatabases
from pyDatabases import gpy, GpyDB
data = os.path.join(os.getcwd(), 'testdata')

*Load test databases:*

In [2]:
fs = [f"{data}\\test_size1000.gdx", f"{data}\\baselinerun.gdx"] # files
ws = gams.GamsWorkspace() 
g2np = gams2numpy.Gams2Numpy(ws.system_directory)
dbs = {'gms1': ws.add_database_from_gdx(fs[0]), 'gms2': ws.add_database_from_gdx(fs[1]),
       'gpy1': GpyDB(db=fs[0],**{'name': 'testdb1'}), 'gpy2': GpyDB(db=fs[1],**{'name': 'testdb2'})}

# ```GpyDB```

In [3]:
db = dbs['gpy1']

*gpyDB* facilitates interaction with GAMS databases from Python using pandas and the Python API from GAMS. This notebook briefly goes through *GpyDB* class.

The class is a database that works on both ```gams.GamsDatabases``` and ```gpyDB.SeriesDB``` databases. It includes a number of methods for arranging/slicing/subsetting/aggregating data. It also includes methods to deal with/work through aliases. Instances of this class can be initialized in three different ways that defines (at least) four attributes:

```python
class GpyDB:
```  
* ```self.series```: A ```SeriesDB``` database.
* ```self.ws```: A ```gams.GamsWorkspace```.
* ```self.work_folder```: The repository for the ```self.ws```.
* ```self.database```: A ```gams.GamsDatabase``` database (opens from ```self.ws```).
* ```self.g2np```: A ```gams2numpy.Gams2Numpy``` instance (used to write/read data). 
* ```self.name```: Name of database.
* ```self.export_settings```: Dictionary used to specify how the database can be pickled. The key 'dropattrs' specifies which attributes are not pickled directly. The key 'data_folder' specifies where to store data. This is used to carry non-pickleable classes with us. The ```self.database``` cannot be pickled directly, but specifying the 'data_folder' writes the ```self.database``` to a .gdx file that is read again when ```self``` is read from the pickle.

In [4]:
db.__dict__

{'ws': <gams.workspace.GamsWorkspace at 0x23e17ecfe50>,
 'work_folder': 'C:\\Users\\sxj477\\AppData\\Local\\Temp\\tmpllgvmrl5',
 'g2np': <gams2numpy.Gams2Numpy at 0x23e17ec1790>,
 'name': 'testdb1',
 'export_settings': {'dropattrs': ['database', 'ws', 'g2np'],
  'data_folder': 'C:\\Users\\sxj477\\Documents\\GitHub\\pyDatabases'},
 'database': <gams.database.GamsDatabase at 0x23e17ec1700>,
 'series': <pyDatabases.gpyDB.gpyDB.SeriesDB at 0x23e17ec17c0>}

The database has a number of base methods implemented to emphasize that the main object of interest is the ```self.series``` database:
* ```self.__iter__```:  Works through ```self.series.__iter__```; thus, iterates through the ```gpy``` symbols stored in this database. 
* ```self.__len__```: Works through ```self.series.__len__```. Returns the number of symbols in this database.
* ```self.__getitem__(self,item)```: Works through ```self.series.__getitem__```; thus, gets the ```gpy``` symbol with name ```item```. If it cannot find 'item', it looks for potential aliases (see ```self.alias``` method for more).
* ```self.__setitem__(self,item,value)```: Works through ```self.series.__setitem__```; thus, it creates a ```gpy``` instance from ```value``` and stores in ```self.series```.
* ```self.__getstate__```: Specifies how to pickle an instance of the class. All attributes not in ```self.export_settings['dropattrs']``` are written to the pickle. The ```self.database``` can be stored alongside the pickled file by not including 'database' in ```self.export_settings['dropattrs']```. In this case ```self.database``` is written to a gdx file in ```self.export_settings['data_folder']``` with the name ```f"{self.name}.gdx"```.
* ```self.__setstate__```: Specifies how a pickled instance of the class can be loaded using pickling. All attributes except three are read from the pickle: The attributes ('ws','g2np','database') are defined by: 
    * ```self.ws``` is initialized from ```self.work_folder```.
    * ```self.g2np``` is then initialized from ```self.ws```. 
    * ```self.database``` is opened from the stored gdx file if 'database' is not in ```self.export_settings['dropattrs']```. Else a new empty database is initialized from ```self.ws```.
    
    If ```self.series``` is not stored in the pickle (by adding it to ```self.export_settings['dropattrs']```), this is read from the ```self.database``` attribute.

### 1: Initialization

The ```GpyDB``` class can be initialized by specifying a file (```pickle_path```) or a database. The format of the database may vary, and various kwargs adjusts the initialization.

#### 1.1 From pickle

Init from a pickle we can use two keyword arguments ('ws', 'alias'). The 'ws' argument specifies which workspace to initialize the ```self.database``` from. The 'ws' argument can either be ```None```, ```str``` (repository for workspace), ```gams.GamsWorkspace``` (use this). The 'alias' argument is always used. See ```self.alias``` methods for more.

In [5]:
ws_options = [None, os.getcwd(), gams.GamsWorkspace()] # open from None, string, or GamsWorkspace
db = GpyDB(pickle_path = data+'\\testdb1', ws =ws_options[0])

#### 1.2 From db

The ```db``` argument can be a couple of different types. As a dictionary we initialize like the pickle, and thus only adjusts for the workspace/alias arguments:

In [6]:
db =GpyDB(db = db.__dict__, ws = ws_options[1]) # identical to pickle, but opens in a new workspace

We can also initialize with a ```GpyDB```. This also only use the ws/alias arguments:

In [7]:
db = GpyDB(db = db, ws = ws_options[2]) # identical to pickle, but opens in a new workspace

Finally, the fallback option goes through the steps:
1. Initialize ```self.ws``` from 'ws' argument. 
2. Initialize ```self.g2np``` from ```self.ws```.
3. Set ```self.name``` from ```**kwargs```. Default is 'rname'. We use a versionized name that depends on what ```self.ws``` we are working in to make sure that names are unique.
4. Set ```self.export_settings``` with a default of ```self.export_settings['dropattrs'] = ['database,'ws','g2np']```. We can adjust the ```self.export_settings['data_folder']``` with kwargs.
5. The 'db' options is used:
    * If 'db' is ```None```: Initialize ```self.database``` as an empty database with name ```self.name```.
    * If 'db' is ```str```: Initialize ```self.database``` from gdx file 'db' and set name to ```self.name```.
    * If 'db' is ```gams.GamsDatabase``` : Initialize ```self.database```from this 'db' and set name to ```self.name```.
    * If 'db' is ```GpyDB```: Initialize ```self.database``` from ```GpyDB.database```and set name to ```self.name```.
6. Initialize ```self.series``` as a ```Database.SeriesDB``` database using the source ```self.database```.
7. Update alias (see ```self.alias``` method).

Examples:

*Initialize empty, set name and data folder:*

In [8]:
db = GpyDB(**{'name': 'emptydb','data_folder': os.getcwd()})
db.__dict__

{'ws': <gams.workspace.GamsWorkspace at 0x23e268acbb0>,
 'work_folder': 'C:\\Users\\sxj477\\AppData\\Local\\Temp\\tmpijwhdjlr',
 'g2np': <gams2numpy.Gams2Numpy at 0x23e268ee3a0>,
 'name': 'emptydb',
 'export_settings': {'dropattrs': ['database', 'ws', 'g2np'],
  'data_folder': 'C:\\Users\\sxj477\\Documents\\GitHub\\pyDatabases'},
 'database': <gams.database.GamsDatabase at 0x23e268ee760>,
 'series': <pyDatabases.gpyDB.gpyDB.SeriesDB at 0x23e268ee0a0>}

*Initialize from str/gdx:*

In [9]:
db = GpyDB(db=fs[0], **{'name':'fromgdx'})
db.__dict__

{'ws': <gams.workspace.GamsWorkspace at 0x23e268e1f10>,
 'work_folder': 'C:\\Users\\sxj477\\AppData\\Local\\Temp\\tmpc4uk3_k7',
 'g2np': <gams2numpy.Gams2Numpy at 0x23e263f3c10>,
 'name': 'fromgdx',
 'export_settings': {'dropattrs': ['database', 'ws', 'g2np'],
  'data_folder': 'C:\\Users\\sxj477\\Documents\\GitHub\\pyDatabases'},
 'database': <gams.database.GamsDatabase at 0x23e263f3dc0>,
 'series': <pyDatabases.gpyDB.gpyDB.SeriesDB at 0x23e263f32e0>}

*Initialize from gams.GamsDatabase:*

In [10]:
db = GpyDB(db = db.database, **{'name': 'fromgams'})
db.__dict__

{'ws': <gams.workspace.GamsWorkspace at 0x23e268f55e0>,
 'work_folder': 'C:\\Users\\sxj477\\AppData\\Local\\Temp\\tmpmdvwyhq7',
 'g2np': <gams2numpy.Gams2Numpy at 0x23e268e18e0>,
 'name': 'fromgams',
 'export_settings': {'dropattrs': ['database', 'ws', 'g2np'],
  'data_folder': 'C:\\Users\\sxj477\\Documents\\GitHub\\pyDatabases'},
 'database': <gams.database.GamsDatabase at 0x23e268ac910>,
 'series': <pyDatabases.gpyDB.gpyDB.SeriesDB at 0x23e263f30d0>}

### 2: Methods

To keep the database relatively small and efficient we build functionality from other classes (```DbWheels_```). The core functionality in the ```GpyDB``` is limited to the following:
* **Properties:**
    * ```self.symbols```: Returns the dictionary ```self.series.database``` with ```gpy``` symbols; keys = symbol.name, value = symbol.
    * ```self.alias_dict```: Returns a dictionary with mapping from sets to its aliases. Key = set, value = list of aliases.
    * ```self.alias_dict0```: As ```self.alias_dict```, with the set itself added to the list in values. 
    * ```self.alias_notin_db```: Returns ```set``` of aliases that are not in the database itself.
* **Basic methods:**
    * ```self.get(self,item)```: Returns the ```self.series.database['item'].vals``` (pandas like representation). If 'item' is not in the database it searches for an alias.
    * ```self.getTypes(self,types)```: Returns ```self.symbols``` for symbols that are of type 'types'; should be iterative and specify ```gpy.type``` types.
    * ```self.copy(self,dropattrs=['database'],**kwargs)```: Return copy of the database without attributes in 'dropattrs'; kwargs is used to modify attributes.
    * ```self.domains_unique(self,x)```: Returns list of sets that a symbol 'x' is defined over. If it is defined over aliased sets, this only returns the parent set.
    * ```self.vardom(self,set_,types=['parameter','variable'])```: Returns a dictionary with key = set_ (includes aliases), and values = list of symbols of type 'types' that have 'set_i' in its domains.
    * ```self.merge_internal(self,merge=True)```: Merges ```self.series``` into ```self.database```.  If 'merge' is set to ```False``` symbols are replaced in ```self.database```.

*Examples:*

In [11]:
db.symbols

{'alias_set': <pyDatabases.gpyDB._database.gpy at 0x23e268f5be0>,
 'alias_map2': <pyDatabases.gpyDB._database.gpy at 0x23e268f59a0>,
 'i': <pyDatabases.gpyDB._database.gpy at 0x23e268f56a0>,
 'j': <pyDatabases.gpyDB._database.gpy at 0x23e268f5910>,
 'alias_': <pyDatabases.gpyDB._database.gpy at 0x23e27921400>,
 'map': <pyDatabases.gpyDB._database.gpy at 0x23e268f5940>,
 'var': <pyDatabases.gpyDB._database.gpy at 0x23e268f5730>,
 'var1d': <pyDatabases.gpyDB._database.gpy at 0x23e268f5eb0>,
 'param': <pyDatabases.gpyDB._database.gpy at 0x23e268f5d30>,
 'scalar': <pyDatabases.gpyDB._database.gpy at 0x23e268f5fd0>,
 'pscalar': <pyDatabases.gpyDB._database.gpy at 0x23e268f5d00>,
 'subset': <pyDatabases.gpyDB._database.gpy at 0x23e27921280>}

In [12]:
db.get('var')

i  j  
1  1      10.0
   0      10.0
   2      10.0
   3      10.0
   4      10.0
          ... 
   995    10.0
   996    10.0
   997    10.0
   998    10.0
   999    10.0
Name: level, Length: 1000, dtype: object

In [13]:
db.getTypes(['variable'])

{'var': <pyDatabases.gpyDB._database.gpy at 0x23e268f5730>,
 'var1d': <pyDatabases.gpyDB._database.gpy at 0x23e268f5eb0>}

In [14]:
db_copy = db.copy()
db_copy.__dict__

{'ws': <gams.workspace.GamsWorkspace at 0x23e268f55e0>,
 'work_folder': 'C:\\Users\\sxj477\\AppData\\Local\\Temp\\tmpmdvwyhq7',
 'g2np': <gams2numpy.Gams2Numpy at 0x23e263f3d00>,
 'name': 'fromgams_0',
 'export_settings': {'dropattrs': ['database', 'ws', 'g2np'],
  'data_folder': 'C:\\Users\\sxj477\\Documents\\GitHub\\pyDatabases'},
 'database': <gams.database.GamsDatabase at 0x23e159f2a30>,
 'series': <pyDatabases.gpyDB.gpyDB.SeriesDB at 0x23e263f34c0>}

In [15]:
db.domains_unique('var')

['i', 'j']

In [16]:
db.vardom('j',types=['parameter','variable','mapping']) # symbols defined over the domain 'j'

{'j': ['map', 'var', 'param']}

In [17]:
db.merge_internal()

### 3: Alias methods

We keep track of aliased symbols using a few database entries. The two-dimensional multiindex 'alias_' maps from sets that are aliased ('alias_set'), to the aliases ('alias_map2'). These three symbols are always carried along and cannot be changed. If no aliases are provided, these are simply empty sets/mappings:

In [18]:
db.get('alias_')

MultiIndex([], names=['alias_set', 'alias_map2'])

Aliases are updated using the ```self.update_alias``` method that takes a suitable multiindex as input. This, for instance, defines an alias to the set 'j' called 'jj':

In [19]:
db.update_alias(pd.MultiIndex.from_tuples([('j','jj')]))
db.get('alias_')

MultiIndex([('j', 'jj')],
           names=['alias_set', 'alias_map2'])

The ```self.alias_dict```, ```self.alias_dict0``` methods returns this set as mapping (dict):

In [20]:
db.alias_dict,db.alias_dict0

({'j': Index(['jj'], dtype='object', name='alias_map2')},
 {'j': Index(['j', 'jj'], dtype='object', name='alias_map2')})

The ```self.alias(x,index_=0)``` method uses this to return aliased symbols from the index 'index_' referring to its element in ```self.alias_dict0```:

In [21]:
(db.alias('j'), db.alias('j',1))

('j', 'jj')

If we use the method on a set that is not aliased, and ```index_0=0``` the method returns the set itself:

In [22]:
db.alias('i')

'i'

If we use this on a variable or similar, we get a ``` TypeError```:

In [23]:
v = 'var'
try:
    db.alias(v)
except TypeError:
    print(f"{v} is not aliased")

var is not aliased
